# 02_EXPLORE_LOANS.ipynb
======================

Objetivo: Explorar y entender el dataset de préstamos
- ¿Cuántos préstamos hay?
- ¿Por qué hay múltiples filas por préstamo? (snapshots)
- ¿Qué información de estado del préstamo tenemos?
- ¿Cómo identificar el estado más reciente de cada préstamo?

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:

base_path = Path.cwd().parent if 'analisis_adhoc' in str(Path.cwd()) else Path.cwd()
data_path = base_path / 'data' / 'raw'

loans = pd.read_csv(data_path / 'AE_challenge_loans.csv')

In [ ]:

print("="*80)
print("📊 DATASET: LOANS")
print("="*80)

# 1. ¿Cuántas filas hay?
print(f"\n1️⃣ Total de filas en el CSV: {len(loans):,}")

# 2. ¿Qué columnas tengo?
print(f"\n2️⃣ Columnas disponibles:")
for col in loans.columns:
    print(f"   - {col}")

# 3. Ver primeros registros
print(f"\n3️⃣ Primeras 10 filas:")
display(loans.head(10))

# 4. ¿Hay datos faltantes?
print(f"\n4️⃣ Valores nulos por columna:")
print(loans.isnull().sum())


In [ ]:

# 5. 🔍 PROBLEMA: ¿Cuántos préstamos ÚNICOS hay?
print(f"\n5️⃣ ANÁLISIS DE DUPLICADOS:")
unique_loans = loans['loan_id'].nunique()
total_rows = len(loans)
print(f"   - Préstamos únicos (loan_id): {unique_loans:,}")
print(f"   - Total de filas: {total_rows:,}")
print(f"   - Diferencia: {total_rows - unique_loans:,} filas adicionales")
print(f"   - Promedio de filas por préstamo: {total_rows / unique_loans:.1f}")


In [ ]:

# 6. ¿Por qué hay múltiples filas? Ver ejemplo
ejemplo_loan = loans['loan_id'].iloc[0]
ejemplo_df = loans[loans['loan_id'] == ejemplo_loan].sort_values('limit_month')

print(f"\n6️⃣ EJEMPLO: loan_id = {ejemplo_loan}")
print(f"   Este préstamo aparece {len(ejemplo_df)} veces en el CSV")
print("\n   Filas de este préstamo (ordenadas por limit_month):")
display(ejemplo_df[['loan_id', 'limit_month', 'disbursed_date', 'days_past_due', 'outstanding_balance', 'status']])

print(f"\n💡 INTERPRETACIÓN:")
print(f"   - Cada fila = 1 SNAPSHOT mensual del estado del préstamo")
print(f"   - 'limit_month' = mes en que se tomó la foto del préstamo")
print(f"   - Los valores de 'days_past_due' y 'outstanding_balance' cambian mes a mes")

# 7. ¿Cuántos snapshots hay por mes?
loans['limit_month'] = pd.to_datetime(loans['limit_month'])
print(f"\n7️⃣ SNAPSHOTS por mes:")
print(loans['limit_month'].value_counts().sort_index())


In [ ]:

# 8. ¿Cuándo se desembolsaron los préstamos? (vintage)
loans['disbursed_date'] = pd.to_datetime(loans['disbursed_date'])
loans['vintage_month'] = loans['disbursed_date'].dt.to_period('M')

print(f"\n8️⃣ VINTAGES (mes de desembolso):")
print(loans.groupby('vintage_month')['loan_id'].nunique().sort_index())

# 9. ¿Qué estados de préstamo existen?
print(f"\n9️⃣ ESTADOS del préstamo:")
print(loans['status'].value_counts())

# 10. ¿Distribución de días de atraso (DPD)?
print(f"\n🔟 DÍAS DE ATRASO (days_past_due):")
print(f"   - Promedio: {loans['days_past_due'].mean():.1f} días")
print(f"   - Mediana: {loans['days_past_due'].median():.1f} días")
print(f"   - Máximo: {loans['days_past_due'].max():.0f} días")

print(f"\n   Distribución por rangos:")
print(f"   - 0 días (al día): {(loans['days_past_due'] == 0).sum():,} registros")
print(f"   - 1-30 días: {((loans['days_past_due'] > 0) & (loans['days_past_due'] <= 30)).sum():,} registros")
print(f"   - 31-60 días: {((loans['days_past_due'] > 30) & (loans['days_past_due'] <= 60)).sum():,} registros")
print(f"   - 61-90 días: {((loans['days_past_due'] > 60) & (loans['days_past_due'] <= 90)).sum():,} registros")
print(f"   - 90+ días: {(loans['days_past_due'] > 90).sum():,} registros")


In [ ]:

print("\n" + "="*80)
print("✅ CONCLUSIONES")
print("="*80)
print(f"""
Total filas: {len(loans):,}
Préstamos únicos: {unique_loans:,}
Snapshots por préstamo: ~{total_rows / unique_loans:.0f}

- PROBLEMA IDENTIFICADO:
Cada préstamo tiene múltiples filas (snapshots mensuales).
Para análisis financiero necesitamos SOLO 1 fila por préstamo.

- SOLUCIÓN (modelado c. dbt):
Tomar el snapshot más reciente de cada préstamo (max limit_month).
""")